In [1]:
import numpy as np
import pandas as pd
import csv
from tempfile import mkdtemp
import os.path as path
from scipy.sparse import csr_matrix, isspmatrix
import json
import io
from gensim.models import Word2Vec
import re
from time import gmtime, strftime
from datetime import datetime
import shlex, subprocess
import time
import glob, os
from krovetzstemmer import Stemmer
import string
from collections import Counter
import itertools
import random
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

C:\Users\nroy0\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def prepare_question_text(question_text):
    if(question_text.startswith('<--')):
        question_text = ''
    question_text = question_text.replace('?','')
    question_text = question_text.replace('</P>','')
    question_text = question_text.replace('<P>','')
    question_text = question_text.replace('!','')
    question_text = question_text.replace('#','')#remove ?
    question_text = question_text.replace("'",'')
    question_text = question_text.replace('"','')
    question_text = question_text.replace('-',' ')
    question_text = question_text.replace('(','')
    question_text = question_text.replace(')','')
    question_text = question_text.replace('[','')
    question_text = question_text.replace(']','')
    question_text = question_text.replace('{','')
    question_text = question_text.replace('}','')
    question_text = question_text.replace(',','')
    question_text = question_text.replace('.','')
    question_text = question_text.replace('&',' and ')
    question_text = question_text.replace(':','')
    question_text = question_text.replace('>','')#error in dataset
    question_text = question_text.replace('<','')
    question_text = question_text.replace(';','')
    question_text = question_text.replace('/',' ')
    question_text = question_text.replace('\n','')

    question_text = question_text.replace('SUMMARY','')
    question_text = question_text.replace('END SUMMARY','')
    
    

    return re.sub(' +',' ', question_text.lower())

In [4]:
stop = json.load(open(r'C:\Users\nroy0\Downloads\Thesis\glove.6B\SMARTstop.json'))

In [5]:

queryDF = pd.DataFrame(columns=['queryNum', 'query'])
i = 0
with open('query.txt', 'w') as file_to_write:
#     file_to_write.write("<parameter>\n")
    with open('Topic', 'r') as file_to_read:
        for line in file_to_read:
            
            if line.startswith("<num>"):
#                 file_to_write.write("<query>\n")
                queryNum = re.search(r'\d+', line)
#                 print( queryNum.group())
#                 print(queryNum.group())
            if line.startswith("<title>"):
                if(line.split("<title>",1)[1][:-1] == ""):
                    queryDF.loc[i] = [int(queryNum.group()), prepare_question_text(next(file_to_read)[:-1])]
                    i = i+1
#                     print(queryNum.group() + ' '+ prepare_question_text(next(file_to_read)[:-1]))
                else:
                    queryDF.loc[i] = [int(queryNum.group()), prepare_question_text(line.split("<title> ",1)[1][:-1])]
                    i = i+1
#                     print( queryNum.group() + ' '+ prepare_question_text(line.split("<title> ",1)[1][:-1]))
#                 file_to_write.write("</query>\n")
    # if it starts with <num>, extract the qnum
            # if it starts with <title>, extract the title
            # break from while loop
    file_to_read.close()
#     file_to_write.write("</parameter>\n")
file_to_write.close()
        # write <query>
        # write the num
        # write the text
# queryDF
        # write </query>

In [6]:
# -*- coding: utf-8 -*-
import re
caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = text.replace("<!","<")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "e.g." in text: text = text.replace("e.g.","e<prd>g<prd>")
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("-->","--><stop>")
    text = text.replace(" <--", "<stop> <--")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [7]:
def binaryEncoding(wordList, qText):
    questionList = qText
    binaryEncoded =np.array([ 1 if i in questionList else 0 for i in wordList])
    return binaryEncoded

In [8]:
def normalise(weights):
    sumW = np.sum(weights)
    normW = weights/sumW
    return normW

In [9]:
def removeStopWords(questionText):
    return [w for w in questionText.split()  if not w in stop]

In [24]:
def candidateTerms(q_id, query,k, alpha):
    #U = words.as_matrix()
    if alpha == 0.1:
        U = embedding01[q_id]
        wordList = words01[q_id]
    elif alpha ==0.01:
        U = embedding001[q_id]
        wordList = words001[q_id]
    else:
        U = embedding001[q_id]
        wordList = words001[q_id]
        
    
#     qText = prepare_question_text(q_id, question)
#     filtered_text = removeStopWords(qText)
    q = binaryEncoding(wordList, query.split())
    tmp = np.matmul(U.T,q)
    candidate = np.matmul(U,tmp)
    index = np.argsort(-candidate)[:k]
    weights = -np.sort(-candidate)[:k]
    normW = normalise(weights)
    expansionTerms = wordList[index]
    pqPlus = dict(zip(expansionTerms,normW))
    return pqPlus

In [11]:
def convertExpansionTerms(question_text, arr):
    onlyUniqueWords = [i for i in list(arr) if i.isalpha()]
#     onlyUniqueWords = np.setdiff1d(onlyWords, question_text.split())
    weightO = normalise([arr[x] for x in onlyUniqueWords])
    testStr = ""
    for item in range(len(onlyUniqueWords)):
        testStr = testStr + str(round(weightO[item],4))+" "+str(onlyUniqueWords[item]) + " "
    return testStr

In [12]:
def queryFormat(lamda, question_text, arr):
    expansionTerms = convertExpansionTerms(question_text, arr)
    filtered_text = removeStopWords(question_text)
    finalQueryText = '<text>#weight( ' + str(lamda)+' #combine ( '+ ' '.join(word for word in filtered_text) + " ) " + str(1 - lamda) + " #weight( " + expansionTerms + ") )</text>"
    return finalQueryText

In [16]:
relevantdoc

['FT924-5307',
 'LA052190-0106',
 'FT933-7438',
 'LA092490-0069',
 'FT942-6645',
 'FT923-14971',
 'FT943-677',
 'FT944-18645',
 'LA061389-0087',
 'FBIS3-41801',
 'FT942-7603',
 'LA070489-0051',
 'LA110389-0072',
 'LA101289-0068',
 'LA071989-0117',
 'FR941107-2-00231',
 'FR940523-0-00045',
 'FBIS4-23032',
 'LA050490-0046',
 'LA110690-0193',
 'LA090689-0027',
 'FT944-3523',
 'FR940202-2-00101',
 'LA012089-0018',
 'LA092689-0080',
 'FT944-18821',
 'LA092689-0119',
 'FBIS4-23025',
 'LA070790-0007',
 'LA032190-0081',
 'LA030690-0009',
 'FBIS4-33295',
 'FR940202-2-00103',
 'FT924-4737',
 'FT932-16160',
 'FT943-7095',
 'FBIS4-22919',
 'FT943-208',
 'FR941202-2-00003',
 'LA041889-0082',
 'FT931-14388',
 'FT923-10165',
 'FT931-10954',
 'FT922-12251',
 'FT941-139',
 'FT933-10469',
 'FT941-879',
 'FR940523-0-00036',
 'FBIS4-23029',
 'FBIS4-22992',
 'FR940523-0-00040',
 'FR941107-2-00230',
 'FR940617-1-00044',
 'FT942-13694',
 'LA101289-0211',
 'LA051190-0185',
 'LA112690-0027',
 'FBIS3-22677',
 '

In [ ]:
# command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\dumpindex.exe" Vol45/index di docno ' + 'FR940126-0-00002'
# args = shlex.split(command_input)
# # p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
# # subprocess.check_output(args)
# docid = subprocess.run(args, stdout=subprocess.PIPE).stdout.decode('utf-8')
# new_command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\dumpindex.exe" Vol45/index dt ' + str(docid) + '> doctext.txt'
# new_args = shlex.split(new_command_input)
# p = subprocess.run(new_args, stderr=subprocess.STDOUT,shell=True)
# # .stdout.decode('utf-8')

In [13]:
os.chdir("C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/Vol45-corpus/collection/corpus/fbis")
# for file in glob.glob("*.*"):
#     print(file)
# glob.glob("*.File")check = 1
check = 1
fbis= {}
parsing = False
fileCount = 0
for file in os.listdir():
    fileCount+=1
#     print(fileCount)
    file_to_read = open(file).readlines()
    for line in file_to_read:
        if line.startswith('<DOCNO> '):
            if check !=1:
                fbis[queryNum.group(1)] =''.join(fbis[queryNum.group(1)])
                fbis[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(fbis[queryNum.group(1)])]))

            queryNum = re.search('<DOCNO> (.*) </DOCNO>', line)

            fbis[queryNum.group(1)] = []
            check = check + 1

        if line.startswith('</TEXT>'):
            parsing = False
        if parsing:
            fbis[queryNum.group(1)].append(line)
        if line.startswith('<TEXT>'):
            parsing = True
fbis[queryNum.group(1)] =''.join(fbis[queryNum.group(1)])
fbis[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(fbis[queryNum.group(1)])]))

In [14]:
os.chdir("C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/Vol45-corpus/collection/corpus/ft")
# for file in glob.glob("*.*"):
#     print(file)
# glob.glob("*.File")check = 1
check = 1
ft= {}
parsing = False
fileCount = 0
for file in os.listdir():
    fileCount+=1
#     print(fileCount)
    file_to_read = io.open(file, errors='ignore').readlines()
    for line in file_to_read:
        if line.startswith('<DOCNO>'):
            if check !=1:
                ft[queryNum.group(1)] =''.join(ft[queryNum.group(1)])
                ft[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(ft[queryNum.group(1)])]))

            queryNum = re.search('<DOCNO>(.*)</DOCNO>', line)

            ft[queryNum.group(1)] = []
            check = check + 1

        if line.startswith('</TEXT>'):
            parsing = False
        if parsing:
            ft[queryNum.group(1)].append(line)
        if line.startswith('<TEXT>'):
            parsing = True
ft[queryNum.group(1)] =''.join(ft[queryNum.group(1)])
ft[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(ft[queryNum.group(1)])]))

In [15]:
os.chdir("C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/Vol45-corpus/collection/corpus/latimes")
# for file in glob.glob("*.*"):
#     print(file)
# glob.glob("*.File")check = 1
check = 1
la= {}
parsing = False
fileCount = 0
for file in os.listdir():
    fileCount+=1
    print(fileCount)
    file_to_read = open(file).readlines()
    for line in file_to_read:
        if line.startswith('<DOCNO> '):
            if check !=1:
                la[queryNum.group(1)] =''.join(la[queryNum.group(1)])
                la[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(la[queryNum.group(1)])]))

            queryNum = re.search('<DOCNO> (.*) </DOCNO>', line)

            la[queryNum.group(1)] = []
            check = check + 1

        if line.startswith('</TEXT>'):
            parsing = False
        if parsing:
            la[queryNum.group(1)].append(line)
        if line.startswith('<TEXT>'):
            parsing = True
la[queryNum.group(1)] =''.join(la[queryNum.group(1)])
la[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(la[queryNum.group(1)])]))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
os.chdir("C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/Vol45-corpus/collection/corpus/fr94")
# for file in glob.glob("*.*"):
#     print(file)
# glob.glob("*.File")check = 1
check = 1
fr= {}
parsing = False
fileCount = 0
for file in os.listdir():
    fileCount+=1
#     print(fileCount)
    file_to_read = open(file).readlines()
    for line in file_to_read:
        if line.startswith('<DOCNO> '):
            if check !=1:
                fr[queryNum.group(1)] =''.join(fr[queryNum.group(1)])
#                 print(fr[queryNum.group(1)])
                fr[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(fr[queryNum.group(1)])]))

            queryNum = re.search('<DOCNO> (.*) </DOCNO>', line)

            fr[queryNum.group(1)] = []
            check = check + 1

        if line.startswith('</TEXT>'):
            parsing = False
        if parsing:
            fr[queryNum.group(1)].append(line)
        if line.startswith('<TEXT>'):
            parsing = True
fr[queryNum.group(1)] =''.join(fr[queryNum.group(1)])
fr[queryNum.group(1)] = list(filter(None, [prepare_question_text(s) for s in split_into_sentences(fr[queryNum.group(1)])]))

In [17]:
tmp1 = dict(fr, **la)
tmp2 = dict(tmp1, **ft)
docdic = dict(tmp2, **fbis)

In [22]:
# embedding01 = {}
# words01 ={}
embedding001 = {}
words001 = {}
    
for queryID in queryDF.queryNum.tolist():
    relevantdoc = []
    startTime = time.time()
    with open(r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\original\output\baselineQL_[1000].indri') as indrifile:
        for output in indrifile:
            if output[:3] == str(queryID):
                doc = re.search(r'Q0 (.*?) ', output).group(1)
                relevantdoc.append(doc)
    embedding001[queryID], words001[queryID] = localExpansion(relevantdoc, 0.001)
    endTime = time.time()
    t_sec = round(endTime - startTime)
    (t_min, t_sec) = divmod(t_sec,60)
    print('QueryID {} took {} minutes and {} seconds'.format(queryID, t_min, t_sec))

QueryID 301 took 1 minutes and 1 seconds
QueryID 302 took 0 minutes and 27 seconds
QueryID 303 took 0 minutes and 33 seconds
QueryID 304 took 0 minutes and 48 seconds
QueryID 305 took 0 minutes and 56 seconds
QueryID 306 took 0 minutes and 46 seconds
QueryID 307 took 1 minutes and 59 seconds
QueryID 308 took 1 minutes and 51 seconds
QueryID 309 took 0 minutes and 51 seconds
QueryID 310 took 0 minutes and 55 seconds
QueryID 311 took 1 minutes and 0 seconds
QueryID 312 took 0 minutes and 1 seconds
QueryID 313 took 1 minutes and 11 seconds
QueryID 314 took 1 minutes and 5 seconds
QueryID 315 took 0 minutes and 47 seconds
QueryID 316 took 0 minutes and 7 seconds
QueryID 317 took 0 minutes and 54 seconds
QueryID 318 took 0 minutes and 54 seconds
QueryID 319 took 1 minutes and 12 seconds
QueryID 320 took 1 minutes and 12 seconds
QueryID 321 took 0 minutes and 50 seconds
QueryID 322 took 0 minutes and 52 seconds
QueryID 323 took 0 minutes and 51 seconds
QueryID 324 took 0 minutes and 37 secon

QueryID 647 took 0 minutes and 39 seconds
QueryID 648 took 1 minutes and 8 seconds
QueryID 649 took 0 minutes and 41 seconds
QueryID 650 took 0 minutes and 42 seconds
QueryID 651 took 0 minutes and 58 seconds
QueryID 652 took 0 minutes and 22 seconds
QueryID 653 took 0 minutes and 32 seconds
QueryID 654 took 0 minutes and 54 seconds
QueryID 655 took 1 minutes and 14 seconds
QueryID 656 took 1 minutes and 0 seconds
QueryID 657 took 0 minutes and 46 seconds
QueryID 658 took 0 minutes and 47 seconds
QueryID 659 took 1 minutes and 0 seconds
QueryID 660 took 0 minutes and 52 seconds
QueryID 661 took 0 minutes and 52 seconds
QueryID 662 took 0 minutes and 36 seconds
QueryID 663 took 0 minutes and 55 seconds
QueryID 664 took 1 minutes and 2 seconds
QueryID 665 took 0 minutes and 43 seconds
QueryID 666 took 0 minutes and 33 seconds
QueryID 667 took 0 minutes and 49 seconds
QueryID 668 took 0 minutes and 55 seconds
QueryID 669 took 1 minutes and 3 seconds
QueryID 670 took 0 minutes and 52 secon

In [72]:
queryDF

,queryNum,query
0,301,international organized crime
1,302,poliomyelitis and post polio
2,303,hubble telescope achievements
3,304,endangered species mammals
4,305,most dangerous vehicles
5,306,african civilian deaths
6,307,new hydroelectric projects
7,308,implant dentistry
8,309,rap and crime
9,310,radio waves and brain cancer


In [27]:
arr = candidateTerms(queryDF['queryNum'][0], queryDF['query'][0] , 50, 0.001)
queryFormat(0.5, queryDF['query'][0], arr)

'<text>#weight( 0.5 #combine ( international organized crime ) 0.5 #weight( 0.0281 organized 0.0281 crime 0.028 drug 0.0243 cases 0.0239 crimes 0.0229 criminal 0.0225 law 0.0224 money 0.0218 economic 0.0217 international 0.0206 drugs 0.0204 corruption 0.0201 illegal 0.0199 activities 0.0197 number 0.0197 order 0.0195 states 0.0195 organs 0.0194 trafficking 0.0194 country 0.0193 public 0.0193 struggle 0.0193 including 0.0191 people 0.019 social 0.019 groups 0.019 police 0.019 fight 0.0189 criminals 0.0189 mafia 0.0189 political 0.0189 measures 0.0188 work 0.0188 system 0.0186 legal 0.0186 countries 0.0185 rights 0.0184 world 0.0183 years 0.0183 case 0.0183 make 0.0183 group 0.0182 office 0.0182 activity 0.0181 control 0.0181 security 0.0181 time 0.018 year 0.018 situation 0.018 internal ) )</text>'

In [19]:
def localExpansion(relevantdoc, alpha):
    p =  [docdic[doc] for doc in relevantdoc]
    text = list(itertools.chain.from_iterable(p))
    tokens = [s.split() for s in text]
    table = str.maketrans('', '', string.punctuation)
    sentenceswithblanks = [[w.translate(table) for w in x if not w in stop] for x in tokens]
    sentences = list(filter(None, sentenceswithblanks))
    model = Word2Vec(sentences, size = 400,  min_count = 1, alpha = alpha )
    model.train(sentences, total_examples = len(sentences), epochs = 80)

    U = model.wv.vectors
    wordList = np.array(model.wv.index2word)
    return U, wordList

In [38]:
def originalLocal(expansion_terms, lamda, X_train, mode, cv):
    if mode == 'train':
        queryFile = 'C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/LocalQueries/train/' + str(cv)+'/query_local_'+str(lamda) + '_' + str(expansion_terms) +'.txt'
    elif mode == 'test':
        queryFile = 'C:/Users/nroy0/Downloads/Thesis/aquaint/Vol45/LocalQueries/test/query_local_'+str(lamda) + '_' + str(expansion_terms) + '_' + str(cv) +'.txt'

     
    with io.open(queryFile, 'w',  encoding="utf-8") as file_to_write:
        file_to_write.write("<parameter>\n")
        for i in range(len(X_train)):
#             print(i)
            file_to_write.write("<query>\n")
            file_to_write.write('<number>'+ str(X_train['queryNum'][i])+'</number>\n')
            question_text =prepare_question_text(X_train['query'][i])
            arr = candidateTerms(X_train['queryNum'][i], question_text , expansion_terms, 0.001)
            finalQueryText = queryFormat(lamda, question_text, arr)
            file_to_write.write(finalQueryText + '\n')
            with open(r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\original\output\baselineQL_[1000].indri') as indrifile:
                for output in indrifile:
                    if output[:3] == str(X_train['queryNum'][i]):
                        doc = re.search(r'Q0 (.*?) ', output).group(1)
                        file_to_write.write('<workingSetDocno>'+doc+ '</workingSetDocno>\n')
            indrifile.close()
            file_to_write.write("</query>\n")

#         file_to_read.close()
        file_to_write.write("</parameter>\n")
    file_to_write.close()
    return queryFile

In [31]:
os.chdir("C:/Users/nroy0/Downloads/Thesis/aquaint/")
kf = KFold( n_splits = 5, shuffle = True)
cv = 0
queryTestindex = {}
for train_index,test_index in kf.split(queryDF):
    cv = cv+1
    X_train, X_test = queryDF.loc[train_index].reset_index(), queryDF.loc[test_index].reset_index()
    queryTestindex[cv]= test_index
    for expansion_terms in [ 50, 100, 200]:
        for lamda in [  0.3, 0.45, 0.6, 0.75]: 
            startTime = time.time()
            queryTrainFile = originalLocal(expansion_terms, lamda, X_train, 'train', cv)
            
#             queryTestFile = originalGlobal(expansion_terms, lamda, X_test, 'test', cv)
            endTime = time.time()
            t_sec = round(endTime - startTime)
            (t_min, t_sec) = divmod(t_sec,60)
            print('Query File for Lamda {} and expansion terms {} created, time taken {} minutes and {} seconds'.format(lamda, expansion_terms, t_min, t_sec))
            for mu in [ 500, 750, 1000, 1500, 2000 ]:
                startTime = time.time()
                outputFile = str(cv)+'/QL_['+str(mu)+'_'+str(lamda)+'_'+str(expansion_terms)+'_'+ str(0.001)+'].indri'
                command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" '+str(queryTrainFile)+' -index=Vol45/index -count=1000 -trecFormat=true -queryOffset=-1 -rule=method:dirichlet,mu:' +str(mu)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/original/local/'+str(outputFile) 
                args = shlex.split(command_input)
                p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
                endTime = time.time()
                t_sec = round(endTime - startTime)
                (t_min, t_sec) = divmod(t_sec,60)
                print("Mu {} complete, time take {} min and {} sec".format(mu, t_min, t_sec ))
        print("Cross Validation {} Complete".format(cv))

Query File for Lamda 0.3 and expansion terms 50 created, time taken 8 minutes and 21 seconds
Mu 500 complete, time take 0 min and 26 sec
Mu 750 complete, time take 0 min and 26 sec
Mu 1000 complete, time take 0 min and 25 sec
Mu 1500 complete, time take 0 min and 25 sec
Mu 2000 complete, time take 0 min and 25 sec
Query File for Lamda 0.45 and expansion terms 50 created, time taken 8 minutes and 20 seconds
Mu 500 complete, time take 0 min and 27 sec
Mu 750 complete, time take 0 min and 27 sec
Mu 1000 complete, time take 0 min and 27 sec
Mu 1500 complete, time take 0 min and 27 sec
Mu 2000 complete, time take 0 min and 29 sec
Query File for Lamda 0.6 and expansion terms 50 created, time taken 8 minutes and 39 seconds
Mu 500 complete, time take 0 min and 29 sec
Mu 750 complete, time take 0 min and 26 sec
Mu 1000 complete, time take 0 min and 27 sec
Mu 1500 complete, time take 0 min and 27 sec
Mu 2000 complete, time take 0 min and 27 sec
Query File for Lamda 0.75 and expansion terms 50 cr

Mu 750 complete, time take 0 min and 28 sec
Mu 1000 complete, time take 0 min and 29 sec
Mu 1500 complete, time take 0 min and 28 sec
Mu 2000 complete, time take 0 min and 28 sec
Query File for Lamda 0.6 and expansion terms 50 created, time taken 9 minutes and 11 seconds
Mu 500 complete, time take 0 min and 29 sec
Mu 750 complete, time take 0 min and 28 sec
Mu 1000 complete, time take 0 min and 28 sec
Mu 1500 complete, time take 0 min and 28 sec
Mu 2000 complete, time take 0 min and 28 sec
Query File for Lamda 0.75 and expansion terms 50 created, time taken 9 minutes and 8 seconds
Mu 500 complete, time take 0 min and 29 sec
Mu 750 complete, time take 0 min and 28 sec
Mu 1000 complete, time take 0 min and 28 sec
Mu 1500 complete, time take 0 min and 27 sec
Mu 2000 complete, time take 0 min and 28 sec
Cross Validation 3 Complete
Query File for Lamda 0.3 and expansion terms 100 created, time taken 9 minutes and 12 seconds
Mu 500 complete, time take 0 min and 46 sec
Mu 750 complete, time t

Mu 2000 complete, time take 0 min and 27 sec
Query File for Lamda 0.75 and expansion terms 50 created, time taken 9 minutes and 35 seconds
Mu 500 complete, time take 0 min and 29 sec
Mu 750 complete, time take 0 min and 27 sec
Mu 1000 complete, time take 0 min and 26 sec
Mu 1500 complete, time take 0 min and 31 sec
Mu 2000 complete, time take 0 min and 33 sec
Cross Validation 5 Complete
Query File for Lamda 0.3 and expansion terms 100 created, time taken 10 minutes and 10 seconds
Mu 500 complete, time take 0 min and 55 sec
Mu 750 complete, time take 0 min and 52 sec
Mu 1000 complete, time take 0 min and 54 sec
Mu 1500 complete, time take 0 min and 52 sec
Mu 2000 complete, time take 0 min and 52 sec
Query File for Lamda 0.45 and expansion terms 100 created, time taken 9 minutes and 47 seconds
Mu 500 complete, time take 0 min and 54 sec
Mu 750 complete, time take 0 min and 52 sec
Mu 1000 complete, time take 0 min and 52 sec
Mu 1500 complete, time take 0 min and 52 sec
Mu 2000 complete, t

In [54]:

path = r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\original\local\2'
filenames = glob.glob(path + "/*.csv")
dfs = []
count = 0
for filename in filenames:
    match = re.search(r'\w\w\_\[\d\.?\d*\_\d\.?\d*\_\d+\_\d\.\d*\]', filename)
    count = count + 1
#     print(count)
    colname = match.group()
    df = pd.read_table(filename,sep='\t', index_col=None, header=None, usecols=[2], skiprows =1, names =  [colname] , quoting=csv.QUOTE_NONE)
    dfs.append(df)
resultQL = pd.concat(dfs, axis=1, ignore_index = False)

resultQL['index']=[ 'num_q',
'num_ret',
'num_rel',
'num_rel_ret',
'map',
'gm_map',
'Rprec',
'bpref',
'recip_rank',
'iprec_at_recall_0.00',
'iprec_at_recall_0.10',
'iprec_at_recall_0.20',
'iprec_at_recall_0.30',
'iprec_at_recall_0.40',
'iprec_at_recall_0.50',
'iprec_at_recall_0.60',
'iprec_at_recall_0.70',
'iprec_at_recall_0.80',
'iprec_at_recall_0.90',
'iprec_at_recall_1.00',
'P_5',
'P_10',
'P_15',
'P_20',
'P_30',
'P_100',
'P_200',
'P_500',
'P_1000',
'recall_5',
'recall_10',
'recall_15',
'recall_20',
'recall_30',
'recall_100',
'recall_200',
'recall_500',
'recall_1000',
'infAP',
'gm_bpref',
'Rprec_mult_0.20',
'Rprec_mult_0.40',
'Rprec_mult_0.60',
'Rprec_mult_0.80',
'Rprec_mult_1.00',
'Rprec_mult_1.20',
'Rprec_mult_1.40',
'Rprec_mult_1.60',
'Rprec_mult_1.80',
'Rprec_mult_2.00',
'utility',
'11pt_avg',
'binG',
'G',
'ndcg',
'ndcg_rel',
'Rndcg',
'ndcg_cut_5',
'ndcg_cut_10',
'ndcg_cut_15',
'ndcg_cut_20',
'ndcg_cut_30',
'ndcg_cut_100',
'ndcg_cut_200',
'ndcg_cut_500',
'ndcg_cut_1000',
'map_cut_5',
'map_cut_10',
'map_cut_15',
'map_cut_20',
'map_cut_30',
'map_cut_100',
'map_cut_200',
'map_cut_500',
'map_cut_1000',
'relative_P_5',
'relative_P_10',
'relative_P_15',
'relative_P_20',
'relative_P_30',
'relative_P_100',
'relative_P_200',
'relative_P_500',
'relative_P_1000',
'success_1',
'success_5',
'success_10',
'set_P',
'set_relative_P',
'set_recall',
'set_map',
'set_F',
'num_nonrel_judged_ret']
resultQL = resultQL.set_index('index')
# resultQL

In [55]:
resultQL.loc['ndcg_cut_10'].idxmax(axis=1)

'QL_[1000_0.45_100_0.1]'

In [56]:
# queryTestindex[cv]= test_index
cv =2
X_test = queryDF.loc[queryTestindex[cv]].reset_index()
for expansion_terms in [ 100]:
    for lamda in [ 0.45]: 
        startTime = time.time()
#             queryTrainFile = originalGlobal(expansion_terms, lamda, X_train, 'train', cv)

        queryTestFile = originalLocal(expansion_terms, lamda, X_test, 'test', cv)
        endTime = time.time()
        t_sec = round(endTime - startTime)
        (t_min, t_sec) = divmod(t_sec,60)
        print('Query File for Lamda {} and expansion terms {} created, time taken {} minutes and {} seconds'.format(lamda, expansion_terms, t_min, t_sec))
        for mu in [ 1000]:
            startTime = time.time()
            outputFile = 'QL_['+str(mu)+'_'+str(lamda)+'_'+str(expansion_terms)+'_'+str(cv) + '].indri'
            command_input= r'"C:\Users\nroy0\Downloads\Thesis\Lemur\bin\IndriRunQuery.exe" '+str(queryTestFile)+' -index=Vol45/index -count=1000 -trecFormat=true -queryOffset=-1 -rule=method:dirichlet,mu:' +str(mu)+' -stemmer.name=krovetz > C:/Users/nroy0/Downloads/Thesis/trec_eval.9.0/original/local/test/'+str(outputFile) 
            args = shlex.split(command_input)
            p = subprocess.run(args, stderr=subprocess.STDOUT,shell=True)
            endTime = time.time()
            t_sec = round(endTime - startTime)
            (t_min, t_sec) = divmod(t_sec,60)
            print("Mu {} complete, time take {} min and {} sec".format(mu, t_min, t_sec ))

Query File for Lamda 0.45 and expansion terms 100 created, time taken 2 minutes and 24 seconds
Mu 1000 complete, time take 0 min and 13 sec


In [57]:

path = r'C:\Users\nroy0\Downloads\Thesis\trec_eval.9.0\original\local\test'
filenames = glob.glob(path + "/*.csv")
dfs = []
count = 0
for filename in filenames:
    match = re.search(r'\w\w\_\[\d\.?\d*\_\d\.?\d*\_\d+\_\d+\]', filename)
    count = count + 1
#     print(count)
    colname = match.group()
    df = pd.read_table(filename,sep='\t', index_col=None, header=None, usecols=[2], skiprows =1, names =  [colname] , quoting=csv.QUOTE_NONE)
    dfs.append(df)
resultQLTest = pd.concat(dfs, axis=1, ignore_index = False)

resultQLTest['index']=[ 'num_q',
'num_ret',
'num_rel',
'num_rel_ret',
'map',
'gm_map',
'Rprec',
'bpref',
'recip_rank',
'iprec_at_recall_0.00',
'iprec_at_recall_0.10',
'iprec_at_recall_0.20',
'iprec_at_recall_0.30',
'iprec_at_recall_0.40',
'iprec_at_recall_0.50',
'iprec_at_recall_0.60',
'iprec_at_recall_0.70',
'iprec_at_recall_0.80',
'iprec_at_recall_0.90',
'iprec_at_recall_1.00',
'P_5',
'P_10',
'P_15',
'P_20',
'P_30',
'P_100',
'P_200',
'P_500',
'P_1000',
'recall_5',
'recall_10',
'recall_15',
'recall_20',
'recall_30',
'recall_100',
'recall_200',
'recall_500',
'recall_1000',
'infAP',
'gm_bpref',
'Rprec_mult_0.20',
'Rprec_mult_0.40',
'Rprec_mult_0.60',
'Rprec_mult_0.80',
'Rprec_mult_1.00',
'Rprec_mult_1.20',
'Rprec_mult_1.40',
'Rprec_mult_1.60',
'Rprec_mult_1.80',
'Rprec_mult_2.00',
'utility',
'11pt_avg',
'binG',
'G',
'ndcg',
'ndcg_rel',
'Rndcg',
'ndcg_cut_5',
'ndcg_cut_10',
'ndcg_cut_15',
'ndcg_cut_20',
'ndcg_cut_30',
'ndcg_cut_100',
'ndcg_cut_200',
'ndcg_cut_500',
'ndcg_cut_1000',
'map_cut_5',
'map_cut_10',
'map_cut_15',
'map_cut_20',
'map_cut_30',
'map_cut_100',
'map_cut_200',
'map_cut_500',
'map_cut_1000',
'relative_P_5',
'relative_P_10',
'relative_P_15',
'relative_P_20',
'relative_P_30',
'relative_P_100',
'relative_P_200',
'relative_P_500',
'relative_P_1000',
'success_1',
'success_5',
'success_10',
'set_P',
'set_relative_P',
'set_recall',
'set_map',
'set_F',
'num_nonrel_judged_ret']
resultQLTest = resultQLTest.set_index('index')
# resultQLTest['ndcg_cut_10']

In [58]:
np.mean((resultQLTest.loc['ndcg_cut_10']))

0.42216000000000004

In [37]:
resultQLTest ##0.44276

NameError: name 'resultQLTest' is not defined